In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [57]:
df = pd.read_csv('../Dataset/online_retail_II.csv')

In [58]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  object 
 1   StockCode    1067371 non-null  object 
 2   Description  1062989 non-null  object 
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  object 
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 65.1+ MB


In [60]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer ID    243007
Country             0
dtype: int64

In [61]:
df.shape

(1067371, 8)

In [62]:
df.dropna(subset=['Description'], inplace=True)  ## Dropping rows with null Description

In [63]:
# Step 1: Fill nulls with label "GUEST"
df['Customer ID'] = df['Customer ID'].fillna('GUEST')

# Step 2: Convert to string (so that numeric IDs don't look like floats)
df['Customer ID'] = df['Customer ID'].astype(str)

# Step 3: Convert to categorical (saves memory & makes sense semantically)
df['Customer ID'] = df['Customer ID'].astype('category')

In [64]:
# Converting Invoice to categorical
df['Invoice'] = df['Invoice'].astype('category') 

In [65]:
# Converting InvoiceDate to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [66]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [67]:
count_invalid_quantity = len(df[df['Quantity'] <= 0])
print(f'Number of rows with invalid Quantity: {count_invalid_quantity}')

Number of rows with invalid Quantity: 20261


In [68]:
df = df[df['Quantity']
    > 0]  # Keep only rows with positive Quantity

In [69]:
count_invalid_price = len(df[df['Price'] <= 0])
print(f'Number of rows with invalid Price: {count_invalid_price}')

Number of rows with invalid Price: 1057


In [70]:
df = df[df['Price'] > 0]  # Keep only rows with positive Price

In [71]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,1041671.0,10.963448,1.0,1.0,3.0,10.0,80995.0,126.51493
InvoiceDate,1041671,2011-01-03 16:31:26.403269376,2009-12-01 07:45:00,2010-07-12 10:26:00,2010-12-07 15:33:00,2011-07-24 12:05:00,2011-12-09 12:50:00,NaN
Price,1041671.0,4.077038,0.001,1.25,2.1,4.13,25111.09,51.448979


In [72]:
min_price = df['Price'].min()
print(f"The minimum price is: {min_price}")

The minimum price is: 0.001


In [73]:
outlier_quantity = len(df[df['Quantity'] > 10])
print(f'Number of rows with Quantity > 10: {outlier_quantity}')

Number of rows with Quantity > 10: 259951


In [74]:


# Create a copy of your DataFrame to store the cleaned data
df_cleaned = df.copy()

# --- Remove Outliers from 'Quantity' ---
Q1_quantity = df_cleaned['Quantity'].quantile(0.25)
Q3_quantity = df_cleaned['Quantity'].quantile(0.75)
IQR_quantity = Q3_quantity - Q1_quantity
upper_bound_quantity = Q3_quantity + 1.5 * IQR_quantity

df_cleaned = df_cleaned[df_cleaned['Quantity'] <= upper_bound_quantity]


# --- Remove Outliers from 'Price' ---
Q1_price = df_cleaned['Price'].quantile(0.25)
Q3_price = df_cleaned['Price'].quantile(0.75)
IQR_price = Q3_price - Q1_price
upper_bound_price = Q3_price + 1.5 * IQR_price

df_cleaned = df_cleaned[df_cleaned['Price'] <= upper_bound_price]


# Display the shape of the DataFrame to see the effect of the cleaning
print(f"Original DataFrame shape: {df.shape}")
print(f"Cleaned DataFrame shape: {df_cleaned.shape}")

Original DataFrame shape: (1041671, 8)
Cleaned DataFrame shape: (866736, 8)


In [75]:
df_cleaned.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,866736.0,4.876884,1.0,1.0,3.0,8.0,23.0,4.460413
InvoiceDate,866736,2011-01-04 20:44:41.913108480,2009-12-01 07:45:00,2010-07-13 12:33:00,2010-12-07 18:36:00,2011-07-25 16:57:00,2011-12-09 12:50:00,NaN
Price,866736.0,2.82503,0.001,1.25,2.1,3.75,8.7,2.040362


## Feature Engineering


In [76]:
## Creating a new column 'Sales' = 'Quantity' * 'Price'
df_cleaned['Sales'] = df_cleaned['Quantity'] * df_cleaned['Price']
df_cleaned.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Sales
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
7,489434,21523,FANCY FONT HOME SWEET HOME DOORMAT,10,2009-12-01 07:45:00,5.95,13085.0,United Kingdom,59.5
8,489435,22350,CAT BOWL,12,2009-12-01 07:46:00,2.55,13085.0,United Kingdom,30.6


In [77]:
# Extract the Year, Month, Weekday, and Hour into new columns
df_cleaned['Year'] = df_cleaned['InvoiceDate'].dt.year
df_cleaned['Month'] = df_cleaned['InvoiceDate'].dt.month
df_cleaned['Weekday'] = df_cleaned['InvoiceDate'].dt.day_name()
df_cleaned['Hour'] = df_cleaned['InvoiceDate'].dt.hour
df_cleaned.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Sales,Year,Month,Weekday,Hour
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4,2009,12,Tuesday,7
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,2009,12,Tuesday,7
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,2009,12,Tuesday,7
7,489434,21523,FANCY FONT HOME SWEET HOME DOORMAT,10,2009-12-01 07:45:00,5.95,13085.0,United Kingdom,59.5,2009,12,Tuesday,7
8,489435,22350,CAT BOWL,12,2009-12-01 07:46:00,2.55,13085.0,United Kingdom,30.6,2009,12,Tuesday,7


In [78]:
df_cleaned['Customer ID'].value_counts()

Customer ID
GUEST      205179
17841.0     12166
14911.0      8252
12748.0      6596
14606.0      6210
            ...  
16377.0         0
12487.0         0
12466.0         0
18251.0         0
14603.0         0
Name: count, Length: 5943, dtype: int64

In [79]:
df_cleaned['Customer_Type'] = np.where(df_cleaned['Customer ID'] == 'GUEST', 'unidentified', 'identified')
df_cleaned['Customer_Type'].value_counts()

Customer_Type
identified      661557
unidentified    205179
Name: count, dtype: int64

In [80]:
df_cleaned['Customer_Region'] = np.where(df_cleaned['Country'] == 'United Kingdom', 'Domestic', 'International')
df_cleaned['Customer_Region'].value_counts()


Customer_Region
Domestic         808593
International     58143
Name: count, dtype: int64

In [81]:
df_cleaned['IsCancelled'] = np.where(df_cleaned['Invoice'].astype(str).str.startswith('C'), 1, 0)
df_cleaned['IsCancelled'].value_counts()

IsCancelled
0    866736
Name: count, dtype: int64

## “In this dataset, cancellations are negligible (only 1 row), so cancellation-related analysis was skipped.”

In [82]:
# Check the total count of invoices starting with 'C' in the original df
print(f"Total invoices starting with 'C' in the original DataFrame: {df['Invoice'].astype(str).str.startswith('C').sum()}")

# Display a few examples of these invoices
print("\nExample of invoices starting with 'C':")
print(df[df['Invoice'].astype(str).str.startswith('C')].head())

Total invoices starting with 'C' in the original DataFrame: 1

Example of invoices starting with 'C':
       Invoice StockCode Description  Quantity         InvoiceDate   Price  \
76799  C496350         M      Manual         1 2010-02-01 08:24:00  373.57   

      Customer ID         Country  
76799       GUEST  United Kingdom  


In [83]:
df_cleaned = df_cleaned.drop(columns=['IsCancelled'])
df_cleaned.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Sales,Year,Month,Weekday,Hour,Customer_Type,Customer_Region
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4,2009,12,Tuesday,7,identified,Domestic
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,2009,12,Tuesday,7,identified,Domestic
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,2009,12,Tuesday,7,identified,Domestic
7,489434,21523,FANCY FONT HOME SWEET HOME DOORMAT,10,2009-12-01 07:45:00,5.95,13085.0,United Kingdom,59.5,2009,12,Tuesday,7,identified,Domestic
8,489435,22350,CAT BOWL,12,2009-12-01 07:46:00,2.55,13085.0,United Kingdom,30.6,2009,12,Tuesday,7,identified,Domestic


# Creating column named Basket size by grouping invoice and stockid. then removing its outliers...all this to categorize it in BULK AND SMALL order

In [88]:
# Step 1: Recalculate the number of unique items per invoice
Basket_size_per_invoice = df_cleaned.groupby('Invoice')['StockCode'].nunique()

# Step 2: Remove the outliers from this grouped data using IQR
Q1 = Basket_size_per_invoice.quantile(0.25)
Q3 = Basket_size_per_invoice.quantile(0.75)
IQR = Q3 - Q1
upper_bound = Q3 + 1.5 * IQR

Basket_size_per_invoice_cleaned = Basket_size_per_invoice[Basket_size_per_invoice <= upper_bound]

C:\Users\msi\AppData\Local\Temp\ipykernel_18696\2214028659.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  Basket_size_per_invoice = df_cleaned.groupby('Invoice')['StockCode'].nunique()


In [89]:
# Display the descriptive stats of the cleaned data to see the effect
print("Descriptive stats of cleaned basket sizes:")
print(Basket_size_per_invoice_cleaned.describe())

Descriptive stats of cleaned basket sizes:
count    46176.000000
mean        10.415302
std         12.072415
min          0.000000
25%          0.000000
50%          6.000000
75%         17.000000
max         50.000000
Name: StockCode, dtype: float64


In [ ]:
# Step 3: Map the cleaned basket size back to the main DataFrame
df_cleaned['Unique_Items_Per_Invoice'] = df_cleaned['Invoice'].map(Basket_size_per_invoice_cleaned)

# Step 4: Create the 'Basket_Size' column with the median of the cleaned data (e.g., 6) as the threshold
# You can get the new median from the descriptive stats output
median_cleaned = Basket_size_per_invoice_cleaned.median()
df_cleaned['Basket_Size'] = np.where(df_cleaned['Unique_Items_Per_Invoice'] > median_cleaned, 'bulk', 'small')


In [91]:
# Check the final value counts
print("\nFinal Basket Size distribution:")
print(df_cleaned['Basket_Size'].value_counts())


Final Basket Size distribution:
Basket_Size
bulk     470314
small    396422
Name: count, dtype: int64


In [92]:
df_cleaned.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Sales,Year,Month,Weekday,Hour,Customer_Type,Customer_Region,Unique_Items_Per_Invoice,Basket_Size
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4,2009,12,Tuesday,7,identified,Domestic,4.0,small
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,2009,12,Tuesday,7,identified,Domestic,4.0,small
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,2009,12,Tuesday,7,identified,Domestic,4.0,small
7,489434,21523,FANCY FONT HOME SWEET HOME DOORMAT,10,2009-12-01 07:45:00,5.95,13085.0,United Kingdom,59.5,2009,12,Tuesday,7,identified,Domestic,4.0,small
8,489435,22350,CAT BOWL,12,2009-12-01 07:46:00,2.55,13085.0,United Kingdom,30.6,2009,12,Tuesday,7,identified,Domestic,3.0,small
